In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import gensim
from eunjeon import Mecab
from konlpy.tag import Hannanum
import re

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Embedding,TimeDistributed,  Input, LSTM, GRU,Bidirectional,  Dropout, Conv1D, MaxPool1D

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [3]:
train = pd.read_csv('news_train.csv')
test = pd.read_csv('news_test.csv')

In [4]:
import re
from eunjeon import Mecab
from tqdm.notebook import tqdm

def text_preprocessing(text_list):
    
    token_list = []
    stopwords = ['습니다','됩니다','는다' , '으로', 'null', '에서', '으로' , '다고', 
                 '지만', '된다며', '전년', '라며', '라고', 'nbsp','된다','부터','보다',
                '합니다','세요']
    #tokenizer = Kkma()
    
    tokenizer = Mecab()
    #tokenizer = Hannanum()
    for index, text in enumerate(tqdm(text_list)):
        txt = re.sub('[^가-힣a-z新北韓日上中株美]', ' ', text.lower())
        #txt = re.sub('&nbsp')
        txt = txt.replace('&nbsp', ' ')
        txt = txt.strip()
        
        token = tokenizer.morphs(txt)
        token = [t for t in token if t not in stopwords]
        token = [t for t in token if (len(t) >=2) | (t in ['◆','▶','【','%' , '美', '미','러','북','北' , '韓', '日', '中', '上','新','株'])]
        token = [i.strip() for i in token]
        token_list.append(token)
        
    return token_list



train.loc[:, 'titleToken'] = text_preprocessing(train['title'])
train.loc[:, 'contentToken'] = text_preprocessing(train['content'])


test.loc[:, 'titleToken'] = text_preprocessing(test['title'])
test.loc[:, 'contentToken'] = text_preprocessing(test['content'])


train.tail()

,n_id,date,title,content,ord,info,titleToken,contentToken
118740,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",미 FDA 임상3상 허가 임박. 묻고 따블로 갈 바이오 황제주.,48,1,"[모나리자, 중원, 지분, 양수, 결정]","[미, fda, 임상, 허가, 임박, 바이오, 제주]"
118741,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",똑똑해진 소비자..한국도 이젠 소형차 시대,49,1,"[모나리자, 중원, 지분, 양수, 결정]","[똑똑, 해진, 소비자, 한국, 이젠, 소형차, 시대]"
118742,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",똑똑해진 소비자..한국도 이젠 소형차 시대,50,1,"[모나리자, 중원, 지분, 양수, 결정]","[똑똑, 해진, 소비자, 한국, 이젠, 소형차, 시대]"
118743,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,51,1,"[모나리자, 중원, 지분, 양수, 결정]","[한국, tv, 대중, 인터넷, 연결]"
118744,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,52,1,"[모나리자, 중원, 지분, 양수, 결정]","[한국, tv, 대중, 인터넷, 연결]"


In [5]:
train['text'] = train.apply(lambda x : x['titleToken'] + x['contentToken'], axis = 1)
test['text'] = test.apply(lambda x : x['titleToken'] + x['contentToken'], axis = 1)

train = train.drop(['n_id','ord', 'title', 'content'], axis =1)
test = test.drop(['n_id','ord', 'title', 'content'], axis =1)

train.tail(2)

,date,info,titleToken,contentToken,text
118743,20200117,1,"[모나리자, 중원, 지분, 양수, 결정]","[한국, tv, 대중, 인터넷, 연결]","[모나리자, 중원, 지분, 양수, 결정, 한국, tv, 대중, 인터넷, 연결]"
118744,20200117,1,"[모나리자, 중원, 지분, 양수, 결정]","[한국, tv, 대중, 인터넷, 연결]","[모나리자, 중원, 지분, 양수, 결정, 한국, tv, 대중, 인터넷, 연결]"


##### word2vec Embedding

In [6]:
trainTit = [i.split() for i in list(set(train['titleToken'].apply(lambda x : ' '.join(x))))]
trainCont = [i.split() for i in list(set(train['contentToken'].apply(lambda x : ' '.join(x))))]
testTit = [i.split() for i in list(set(test['titleToken'].apply(lambda x : ' '.join(x))))]
testCont = [i.split() for i in list(set(test['contentToken'].apply(lambda x : ' '.join(x))))]

In [7]:
X = train['text']
y = train['info'].values


#네이버뉴스 데이터를 통해 학습한 모델 사용
w2v_model = gensim.models.Word2Vec.load("NaverNewsW2V.model")

##### Tokenizing

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

X = tokenizer.texts_to_sequences(X)

In [10]:
word_index = tokenizer.word_index
for word, num in word_index.items():
    print(f"{word} -> {num}")
    if num == 10:
        break

코로나 -> 1
종목 -> 2
한국 -> 3
가능 -> 4
이상 -> 5
투자 -> 6
시장 -> 7
공개 -> 8
주식 -> 9
경제 -> 10


In [11]:
maxlen = 650

#Making all news of size maxlen defined above
X = pad_sequences(X, maxlen=maxlen)

vocab_size = len(tokenizer.word_index) + 1

In [12]:
#Function to create weight matrix from word2vec gensim model
def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix

In [13]:
embedding_vectors = get_weight_matrix(w2v_model, word_index)

In [14]:
train[['date','text','info']].head(50)

,date,text,info
0,20200605,"[마감, 코스닥, 기관, 순매도, 데일리, marketpoint, 현재, 코스닥, ...",0
1,20200605,"[마감, 코스닥, 기관, 순매도, 실적, 기반, 저가, 매집, 해야, 급등, 유망주...",1
2,20200605,"[마감, 코스닥, 기관, 순매도, 하이스, 탁론, 선취, 수수료, 최저, 금리, 상...",1
3,20200605,"[마감, 코스닥, 기관, 순매도, 종합, 경제, 정보, 미디어, 데일리, 무단, 전...",0
4,20200626,"[롯데, 공영, tv, 쇼핑, 동행, 세일, 동참, 전국, 소비, 조성, 기여, 예정]",0
5,20200626,"[롯데, 공영, tv, 쇼핑, 동행, 세일, 동참, 데일리, 권오석, 기자, 중소,...",0
6,20200626,"[롯데, 공영, tv, 쇼핑, 동행, 세일, 동참, 대한민국, 동행, 세일, 라이브...",0
7,20200626,"[롯데, 공영, tv, 쇼핑, 동행, 세일, 동참, 권역, 현장, 행사, 오프라인,...",0
8,20200626,"[롯데, 공영, tv, 쇼핑, 동행, 세일, 동참, 이번, 동행, 세일, 롯데, 공...",0
9,20200626,"[롯데, 공영, tv, 쇼핑, 동행, 세일, 동참, 동행, 세일, 기간, 쇼핑, 판...",0


##### model

In [15]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers


saveBestweights = ModelCheckpoint('best.weights', monitor='val_loss', verbose=1, save_best_only=True)
earlyStopping   = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [16]:
X_train, X_val, y_train, y_val = train_test_split(pd.DataFrame(X), y, test_size = 0.3)

print(X_train.shape, X_val.shape)

(83121, 650) (35624, 650)


In [18]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(vocab_size, output_dim = EMBEDDING_DIM, weights=[embedding_vectors], input_length=maxlen, trainable=False))
#model.add(Conv1D(activation='relu', filters=3, kernel_size=3))
#model.add(MaxPool1D())
#LSTM 
model.add(LSTM(units=64))
#model.add(Bidirectional(LSTM(units=16 )))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 650, 200)          7177000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                67840     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 7,244,905
Trainable params: 67,905
Non-trainable params: 7,177,000
_________________________________________________________________


In [20]:
#갑자기 에러뜸,, GPU 문제일듯
model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs=20, callbacks=[earlyStopping, saveBestweights])

Epoch 1/20


UnknownError:  [_Derived_]  Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/StatefulPartitionedCall]] [Op:__inference_train_function_3207]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
%%time
valPred = model.predict(X_val)
y_pred = (valPred >= 0.5).astype("int")
print(classification_report(y_val, y_pred))

In [ ]:
print(classification_report(y_val, y_pred))

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_val, y_pred)

In [ ]:
X_train

In [ ]:
trainCheck = pd.read_csv('news_train.csv')
trainCheck = trainCheck.loc[X_val.index]
trainCheck['pred'] = valPred

In [ ]:
%%time
subLabel = model.predict(testX)
subLabel

##### submit 파일 가져오기

In [ ]:
test['pred'] = subLabel
test

In [ ]:
submit = pd.read_csv('sample_submission.csv')

In [ ]:
trainCheck = pd.read_csv('news_train.csv')
testCheck = pd.read_csv('news_test.csv')
testCheck['pred'] = subLabel
testCheck

##### 제출

In [ ]:
submit['info'] = testCheck['predInfo'].astype(int)
submit.to_csv('submit/1215.csv', index = False)
submit